## Python to Create Tables

In [ ]:
import sqlite3
from google.cloud import bigquery
from google.oauth2 import service_account

# Set up your Google BigQuery credentials and client
service_path = r"C:\Users\Sam Beighle\Documents\ADA"
service_file = r'\wedge-project-sam-beighle-3df5c228fad7.json'
gbq_proj_id = 'wedge-project-sam-beighle'
private_key = service_path + service_file
credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

# Specify the path and name of your new SQLite database
new_db_path = "new_database_1.db"

# Establish a connection to the new SQLite database
db_1 = sqlite3.connect(new_db_path)
cursor = db_1.cursor()

In [ ]:

# Define your BigQuery SQL queries and corresponding table schemas
queries_and_schemas = {
    "sales_by_date_2": {
        "query": """
            SELECT
                EXTRACT(DATE FROM datetime) AS date,
                EXTRACT(HOUR FROM datetime) AS hour,
                SUM(total) AS total_spend,
                COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS num_transactions,
                SUM(CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) AS items
            FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_20*`
            WHERE department NOT IN (0, 15)
            AND (trans_status IS NULL OR trans_status IN (' ', 'V', 'R'))
            GROUP BY date, hour
            ORDER BY date, hour
        """,
        "schema": "date DATE, hour INTEGER, total_spend FLOAT, num_transactions INTEGER, items INTEGER"
    },
    "sales_by_owner_2": {
        "query": """
            SELECT
                DISTINCT card_no AS card_no,
                EXTRACT(YEAR FROM datetime) AS year,
                EXTRACT(MONTH FROM datetime) AS month,
                SUM(total) AS sales,
                COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM datetime) AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) AS transactions,
                SUM(CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) AS items
            FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_2*`
            GROUP BY card_no, year, month
            ORDER BY year, month, card_no
        """,
        "schema": "card_no FLOAT, year INTEGER, month INTEGER, sales FLOAT, transactions INTEGER, items INTEGER"
    },
    "sales_by_product_2": {
        "query": """
            WITH filtered_data AS (
                SELECT
                    transtable.upc,
                    transtable.description,
                    transtable.department AS dept_number,
                    Dept.dept_name,
                    EXTRACT(YEAR FROM transtable.datetime) AS year,
                    EXTRACT(MONTH FROM transtable.datetime) AS month,
                    SUM(total) AS sales,
                    COUNT(DISTINCT CONCAT(CAST(EXTRACT(DATE FROM transtable.datetime) AS STRING), CAST(transtable.register_no AS STRING), CAST(transtable.emp_no AS STRING), CAST(transtable.trans_no AS STRING))) AS transactions,
                    SUM(CASE WHEN transtable.trans_status IN ('V', 'R') THEN -1 ELSE 1 END) AS items
                FROM `wedge-project-sam-beighle.wedge_project_beighle.transArchive_20*` AS transtable
                JOIN `wedge-project-sam-beighle.wedge_project_beighle.department_lookup` AS Dept ON Dept.department = transtable.department
                GROUP BY transtable.upc, transtable.description, transtable.department, Dept.dept_name, year, month
                ORDER BY description, upc, year, month, department, Dept.dept_name
            """,
        "schema": "upc STRING, description STRING, dept_number FLOAT, dept_name STRING, year INTEGER, month INTEGER, sales FLOAT, transactions INTEGER, items INTEGER"
    }
}

In [ ]:
# Loop through queries and insert data into respective SQLite tables
for table_name, data in queries_and_schemas.items():
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({data['schema']})")

    # Fetch data from BigQuery and insert it into the table
    query_job = client.query(data['query'])
    rows = query_job.result()

    for row in rows:
        cursor.execute(f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * len(row))})", row)

# Commit the changes to the new SQLite database
db_1.commit()

# Close the database connection
db_1.close()